# Annual Business Survey 2019
## Group Assessment - Module 08

  *****

## ETL Report
### Introduction

- In this portion, we will dicuss the problems or questions that we are trying to solve or answer. 
    We will identify the sources of our data and describe why the data needs to be transformed.
    
The United States Census Bureau provides access to an extensive database with hundreds of datasets that detail national <br>
  characteristics. The data is stored in a way that can be quite confusing to many people, but there are many resources to <br>
  assist interested researchers with gleaning new insights from the data.
  
For this study, we looked specifically at the Annual Business Survey for **2019** which examines data from **2018**. This survey <br>
  provides information on a variety of characteristics for businesses and business owners.
  
**These are the questions that we are focusing on:**<br>
> 1. When considering the Finance and Insurance Industries as well as the Education Industry, what are the reasons that a owner <br>
      was motivated to create the business?
      
> 2. When considering the highest average pay for each state, are there any noticeable trends when accounting for the Industry, <br>
      the Race of the Owner, or the Gender of the Owner?

> 3. Does the rate of utilizing technology solutions have any correlation with family ownership status?

> 4. What industries are most and least likely to utilize tech solutions in their business?
    
> 5. What questions are business owners most likely to respond to?

> 6. What is the category of responses for business owners to the technical questions?

> 7. Is there a gender gap among business owners based on race?

> 8. 

> 9. 

> 10. 

> 11. 

> 12. 

**Why is it important to transform the data?**<br>

As mentioned previously, the data is stored in a way that can be quite confusing, and it is necessary to perform a variety of<br>
  transformations on it before it can be utilized. Some things to keep in mind is that there are multiple series of "total" categories,<br>
  multiple variables stored in a single column, and the call to the API must contain every needed variable, as well as supporting variables<br>
  in order to get complete and accurate results.
### Data Sources

The data used in this study was gathered directly from the U.S. Census Bureau data API. 

>US Census Bureau. (2021b, October 14). *Annual Business Survey (ABS) APIs.* Census.Gov.<br>
>Retrieved April 22, 2022, from https://www.census.gov/data/developers/data-sets/abs.2019.html

### Extraction

- In this portion we will show how we got our data, show aspects of the data consumption process and indicate the order of the process.
**Step 1 - Import Modules**
> In this step, we are gathering all the modules that will be used in the script. We have also included a helper module that can act <br>
as a variable dictionary for someone seeking to understand the code. <-- **I'm not sure if I should include this actually.**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt

import matplotlib.pyplot as plt
import plotly.express as px

from textwrap import wrap
from utils import explain # Helper for clarifying variable information (for questions about this, type "help(explain)" in a cell)


 ----- HELPER MODULE IMPORTED ------

    For information on the helper module:
        run 'help(explain)' in any cell. 
        
    To directly use:
        run 'explain.what_is()' with no parameters.
    


**Step 2 - Get Variable Tables**
> In this step, we gathered the variables from each of the available datasets, and merged them into a single table with a goal<br>
of understanding what data was available in each table, and if there were any common variables between tables.

> To read this data into a dataframe we used the Pandas **"read_html()"** method which returns a list of all tables that are<br>
that are available at the target source. We then kept only the first two columns from each table and renamed a column<br>
that would cause name conflicts in a merged dataframe.

In [2]:
var_links = [
    'https://api.census.gov/data/2018/abstcb/variables.html',
    'https://api.census.gov/data/2018/abscbo/variables.html',
    'https://api.census.gov/data/2018/abscb/variables.html',
    'https://api.census.gov/data/2018/abscs/variables.html'
]

# Get the tables of variables.
tech_vars = pd.read_html(var_links[0])[0]
owner_vars = pd.read_html(var_links[1])[0]
characteristic_vars = pd.read_html(var_links[2])[0]
company_summary_vars = pd.read_html(var_links[3])[0]

# Keep only two columns from each table and rename columns for merge.
tech_vars = tech_vars.iloc[:,:2
                        ].rename(columns = {'Label': 'Tech Labels'})

owner_vars = owner_vars.iloc[:,:2
                        ].rename(columns = {'Label': 'Owner Labels'})

characteristic_vars = characteristic_vars.iloc[:,:2
                        ].rename(columns = {'Label': 'Characteristic Labels'})

company_summary_vars = company_summary_vars.iloc[:,:2
                        ].rename(columns = {'Label': 'Company Summary Labels'})

**Step 3 - Merge Variable Tables**
> In this step we merged all the variable tables using a full outer merge so that all records would<br>
be retained. We also filled all the null cells that were generated in the merge with "-" so that the<br>
table would be easier to read.

In [3]:
grouped_tables = pd.merge(
    tech_vars, owner_vars, left_on = 'Name', right_on = 'Name', how = 'outer'
)

grouped_tables = pd.merge(
    grouped_tables, characteristic_vars, left_on = 'Name', right_on = 'Name', how = 'outer'
)

grouped_tables = pd.merge(
    grouped_tables, company_summary_vars, left_on = 'Name', right_on = 'Name', how = 'outer'
)

grouped_tables.fillna("-", inplace = True)
grouped_tables = grouped_tables[(~grouped_tables['Name'].str.endswith('_S'))]

grouped_tables = grouped_tables[:-2].sort_values(by = 'Name')
grouped_tables = grouped_tables[(~grouped_tables['Name'].str.contains('variables'))]

**Step 4 - Group Similar Variables**
> This step was necessary because some of the variables referenced the same metric<br>
but were named slightly differently. For example, OWNER_ETH vs. ETH_GROUP. 

> We then ordered the generated table by column and by name to prepare it for<br>
further use.

> We created a smaller table from this generated table that consisted only of<br>
the variables that were present in all datsets. This was done in an effort to<br>
reduce the ambiguity of the datasets and inform a merge of all datasets.

In [4]:
def clean_var_names(var_name):
    """ This function assists with grouping similar variables """
    if 'OWNER_' in var_name:
        var_name = var_name.replace('OWNER_',"")
    elif ('OWN' in var_name and 'CHAR' not in var_name and 'PDEMP' not in var_name):
        var_name = var_name.replace('OWN',"")
    
    if var_name in ['ETH','RACE','SEX','VET']:
        var_name = var_name + "_GROUP"
        
    return var_name

grouped_tables['Common_Vars'] = grouped_tables['Name'].apply(clean_var_names)
grouped_tables = grouped_tables[
    ['Name','Common_Vars','Tech Labels','Owner Labels',
     'Characteristic Labels','Company Summary Labels']
]

grouped_tables.sort_values(
    by = 'Common_Vars').reset_index(drop = True, inplace = True)

# Create smaller dataframe that just contains variables present in all tables.
possible_merge_options = grouped_tables[
    (grouped_tables['Common_Vars'].str.contains("SEX|RACE|ETH|VET")) | 
    (grouped_tables.apply(lambda x: not x.str.contains("-").any(), axis = 1))
].sort_values(by = 'Common_Vars').reset_index(drop = True)

# This list contains all variables that were found in the possible_merge_options.
common_vars = [_.lower() for _ in possible_merge_options.Common_Vars.unique().tolist()]

**The cell below is intended for viewing the generated dataframes.**

In [5]:
#possible_merge_options
#grouped_tables

**Step 5 - Define which variables will be requested from API**
> After reviewing the available end points as well as encountering various errors<br>
or missing information, we settled on a list of variables that we were interested in<br>
and isolated those variables and the tables they were in from our grouped tables. 

In [6]:
vars_of_interest = [
    'NAICS2017',
    'YIBSZFI',
    'SEX',
    'QDESC',
    'NSFSZFI',
    'GEO_ID',
    'RACE_GROUP',
    'BUSCHAR',
    'OWNER_RACE',
    'OWNER_SEX',
    'OWNPDEMP',
    'FIRMPDEMP',
    'OWNCHAR',
    'TECHUSE',
    'RCPPDEMP',
    'PAYANN',
    'EMP'
]

target_subset = grouped_tables[(grouped_tables['Name'].isin(vars_of_interest))]
target_subset = target_subset[['Name','Company Summary Labels','Characteristic Labels','Owner Labels','Tech Labels']]

**Step 6 - Build variable strings for api calls**
> In this step, we used our table of variables and which datasets those variables were available in<br>
and generated end point strings to pass into the api call. We used a for loop to generate a dictionary<br>
of end point strings. We also added the "_LABEL" to any of the endpoints that had that option available.

In [7]:
no_label = [
    'OWNPDEMP_LABEL','GEO_ID_LABEL','FIRMPDEMP_LABEL','STATE_LABEL','PAYANN_LABEL',
    'EMP_LABEL','RCPPDEMP_LABEL',
]
variable_dict = {}
for i,label in enumerate(target_subset.columns[1:]):
    variable_list = []
    for item in target_subset[(target_subset[label] != "-")].Name.tolist():
        variable_list.append(item)
        variable_list.append(f'{item}_LABEL')
    variable_list = [_ for _ in variable_list if _ not in no_label]
    in_table = "NAME," + ",".join(variable_list)
    variable_dict[i] = in_table

**Step 7 - Request Data from API**
> Using the dictionary of variable strings that was created in step 6, we generated links for each of the datasets and<br>
requested the data from the api. For each call to the api, we printed the url that was passed so that we could easily<br>
troubleshoot any errors. The final step was to create a list of all the dataframe that would be iterated through for cleaning them.

In [8]:
links = [
    f'https://api.census.gov/data/2018/abscs?get={variable_dict[0]}&for=state:*',
    f'https://api.census.gov/data/2018/abscb?get={variable_dict[1]}&for=state:*',
    f'https://api.census.gov/data/2018/abscbo?get={variable_dict[2]}&for=us:*&for=QDESC_LABEL=YRACQBUS',
    f'https://api.census.gov/data/2018/abstcb?get={variable_dict[3]}&for=state:*',
    f'https://api.census.gov/data/2018/abscbo?get={variable_dict[2]}&for=state:*&OWNCHAR=CG&NAICS2017=00&QDESC=O02',
    f'https://api.census.gov/data/2018/abscs?get={variable_dict[0]},FIRMPDEMP&for=us:*'  #JAKE ADDED THIS
]

def get_data_frame(url):
    print("If the operation fails, click the link to see the error.")
    print(url,'\n')
    return pd.read_csv(url)

comp_sum_df = get_data_frame(links[0]) # Company Summary
bus_char_df = get_data_frame(links[1]) # Business Characteristics
bus_own_df = get_data_frame(links[2]) # Business Owners (National Level)
bus_tech_df = get_data_frame(links[3]) # Business Tech   
slbo = get_data_frame(links[4]) # Business Owners (State Level)
comp_sum_natl_df = get_data_frame(links[5]) #Company Summary National   JAKE ADDED THIS

If the operation fails, click the link to see the error.
https://api.census.gov/data/2018/abscs?get=NAME,EMP,FIRMPDEMP,GEO_ID,NAICS2017,NAICS2017_LABEL,PAYANN,RACE_GROUP,RACE_GROUP_LABEL,RCPPDEMP,SEX,SEX_LABEL,YIBSZFI,YIBSZFI_LABEL&for=state:* 

If the operation fails, click the link to see the error.
https://api.census.gov/data/2018/abscb?get=NAME,BUSCHAR,BUSCHAR_LABEL,EMP,FIRMPDEMP,GEO_ID,NAICS2017,NAICS2017_LABEL,PAYANN,QDESC,QDESC_LABEL,RACE_GROUP,RACE_GROUP_LABEL,RCPPDEMP,SEX,SEX_LABEL,YIBSZFI,YIBSZFI_LABEL&for=state:* 

If the operation fails, click the link to see the error.
https://api.census.gov/data/2018/abscbo?get=NAME,GEO_ID,NAICS2017,NAICS2017_LABEL,OWNCHAR,OWNCHAR_LABEL,OWNER_RACE,OWNER_RACE_LABEL,OWNER_SEX,OWNER_SEX_LABEL,OWNPDEMP,QDESC,QDESC_LABEL&for=us:*&for=QDESC_LABEL=YRACQBUS 

If the operation fails, click the link to see the error.
https://api.census.gov/data/2018/abstcb?get=NAME,EMP,FIRMPDEMP,GEO_ID,NAICS2017,NAICS2017_LABEL,NSFSZFI,NSFSZFI_LABEL,PAYANN,RACE_GRO

In [9]:
df_collection = [comp_sum_df, bus_char_df, bus_own_df, bus_tech_df, slbo, comp_sum_natl_df]      #JAKE ADDED TO THIS LIST
df_names = ['comp_sum_df', 'bus_char_df', 'bus_own_df', 'bus_tech_df','slbo', 'comp_sum_natl_df']   

### Transformation
> Once we had gathered the datasets that we were interested in, we needed to clean the data<br>
so that it could be useful for analysis. 

> We first created a list of columns that could be dropped. These were present because they were<br> 
required for the actual call in order to get a related column.

> We then iterated through our collection of dataframes and cleaned columns that had artifacts from<br>
the method of the api call. 

> Finally, we renamed some ambiguous columns so that their contents would be more clear.


### Clean DataFrames

In [10]:
drop_list = [
    'race_group','sex','yibszfi','qdesc','buschar',
    'owner_race','owner_sex','us','ownchar'
]

for df in df_collection:
    column_names = [_ for _ in df.columns.tolist()]
    new_column_names = [
        _.replace("[[","")
         .replace('"',"")
         .replace("]","")
         .lower() for _ in column_names
    ]

    df.columns = new_column_names
    df.drop(columns = [_ for _ in new_column_names if ('unnamed' in _ or _ in drop_list)],inplace = True)
    df['name'] = df['name'].apply(lambda x: x.replace("[","").replace('"',""))
    
    if 'sex_label' in df.columns:
        df.rename(columns = {'sex_label': 'gender'}, inplace = True)
    
    if 'owner_sex_label' in df.columns:
        df.rename(columns = {'owner_sex_label': 'gender'}, inplace = True)
    
    if 'naics2017_label' in df.columns:
        df.rename(columns = {'naics2017_label': 'industry'}, inplace = True)
        
    if 'naics2017' in df.columns:
        df.rename(columns = {'naics2017': 'industry_code'}, inplace = True)

### Display some info about dataframes and save data

In [11]:
for i,df in enumerate(df_collection):
    try:
        df.to_csv(f'data/{df_names[i]}.csv', index = False)
    except:
        print("Data directory is not present - skipping save")
        
    print('\n############# NEW DATAFRAME ################')
    print('Displaying column value counts where there are fewer than 10 unique values in the column.')
    print(f'\n---  DataFrame: {df_names[i]} ---------------------')
    print(f'Columns: {", ".join(df.columns.tolist())}\n')
    for column in df:
        unique_values = len(df[column].unique().tolist())
        if (unique_values < 30 and unique_values > 1):
            print(df[column].value_counts())
            print("")
    print('############# END OF DATAFRAME INFO ################\n\n')


############# NEW DATAFRAME ################
Displaying column value counts where there are fewer than 10 unique values in the column.

---  DataFrame: comp_sum_df ---------------------
Columns: name, emp, firmpdemp, geo_id, industry_code, industry, payann, race_group_label, rcppdemp, gender, yibszfi_label, state

00       11454
54        4933
31-33     4553
44-45     4500
62        4485
72        4444
23        4361
48-49     4356
56        4301
42        4220
81        4213
51        4050
53        3881
52        3653
71        3403
61        3297
99        2936
21        2453
11        2310
55        2269
22        2120
Name: industry_code, dtype: int64

Total for all sectors                                                       11454
Professional, scientific, and technical services                             4933
Manufacturing                                                                4553
Retail trade                                                                 4500
Healt

## All Tables Merge
 - We certainly had a substantial amount of difficulty with this merge because it was difficult to find ways<br>
     that the the data could be useful when fully merged. <br><br> 
     In this merge set, we are looking at the level of artificial intelligence in all sectors in Minnesota, <br>
     and how it relates to how long the company has been in business, and whether it is a family owned business.

### Steps:

1. Limit table data to "Total for all sectors".

In [12]:
total_bus_tech_df = bus_tech_df[(bus_tech_df['industry'] == 'Total for all sectors')]
total_bus_own_df = bus_own_df[(bus_own_df['industry'] == 'Total for all sectors')]
total_comp_sum_df = comp_sum_df[(comp_sum_df['industry'] == 'Total for all sectors')]
total_bus_char_df = bus_char_df[(bus_char_df['industry'] == 'Total for all sectors')]
total_slbo = slbo[(slbo['industry'] == 'Total for all sectors')]

2. Further filter the table data and remove unnecessary columns.

In [13]:
robotics_in_mn = total_bus_tech_df[(~total_bus_tech_df['techuse_label'].str.contains('Total')) & (total_bus_tech_df['name'] == 'Minnesota') & (total_bus_tech_df['techuse_label'].str.contains('Artificial'))]
robotics_in_mn = robotics_in_mn[['name','techuse_label','firmpdemp']]

years_in_biz_mn = total_comp_sum_df[(total_comp_sum_df.name == 'Minnesota') & (total_comp_sum_df['yibszfi_label'] != 'All firms') & (total_comp_sum_df['gender'] != 'Total')]
years_in_biz_mn = years_in_biz_mn[['name','gender','yibszfi_label','firmpdemp']].drop_duplicates()

bus_char_mn = total_bus_char_df[(total_bus_char_df.name == 'Minnesota') & (total_bus_char_df['buschar_label'] != 'All firms')& (total_bus_char_df['buschar_label'] != 'Total reporting') & (total_bus_char_df['qdesc_label'] == 'FAMOWN')]
bus_char_mn = bus_char_mn[['name','qdesc_label','buschar_label','firmpdemp']]

total_slbo = total_slbo[(total_slbo.name == 'Minnesota')]
total_slbo = total_slbo[['name','ownchar_label','ownpdemp']]

3. Rename similar columns so that they are meaningfull after the merge.

In [14]:
years_in_biz_mn.rename(columns = {'firmpdemp': 'yib_num_firms'}, inplace = True)
robotics_in_mn.rename(columns = {'firmpdemp': 'robin_mn_num_firms'}, inplace = True)
bus_char_mn.rename(columns = {'firmpdemp': 'bus_char_num_firms'}, inplace = True)

4. Merge tables using outer merge to retain all records.

In [15]:
joined_tables = pd.merge(years_in_biz_mn, robotics_in_mn, left_on = 'name', right_on = 'name', how = 'outer')
joined_tables = pd.merge(joined_tables, bus_char_mn, left_on = 'name', right_on = 'name', how = 'outer')
joined_tables = pd.merge(joined_tables, total_slbo, left_on = 'name', right_on = 'name', how = 'outer')

# Just to display a subset of the data
joined_tables.iloc[[_ for _ in range(1870,1876)]]

,name,gender,yibszfi_label,yib_num_firms,techuse_label,robin_mn_num_firms,qdesc_label,buschar_label,bus_char_num_firms,ownchar_label,ownpdemp
1870,Minnesota,Equally male/female,Firms with less than 2 years in business,0,Artificial Intelligence: Don't know,4105,FAMOWN,Item not reported,2293,Before 1980,3210
1871,Minnesota,Equally male/female,Firms with less than 2 years in business,0,Artificial Intelligence: Don't know,4105,FAMOWN,Not applicable,3158,Before 1980,3210
1872,Minnesota,Equally male/female,Firms with 16 or more years in business,272,Artificial Intelligence: Did not use,95053,FAMOWN,Family-owned,17863,Before 1980,3210
1873,Minnesota,Equally male/female,Firms with 16 or more years in business,272,Artificial Intelligence: Did not use,95053,FAMOWN,Not family-owned,40870,Before 1980,3210
1874,Minnesota,Equally male/female,Firms with 16 or more years in business,272,Artificial Intelligence: Did not use,95053,FAMOWN,Item not reported,2293,Before 1980,3210
1875,Minnesota,Equally male/female,Firms with 16 or more years in business,272,Artificial Intelligence: Did not use,95053,FAMOWN,Not applicable,3158,Before 1980,3210


****
# Analysis and Visualizations
> This section contains the code that was used to further transform the data as needed<br>
for visualizations and build visualizations.

In [16]:
# I suggest wrapping your entire visualization process in a function while we work on integrating. 

# For example. 

def jeds_visuals():
    today = dt.datetime.strftime(dt.datetime.today().date(), '%b %d, %Y')
    state_code_path = 'https://raw.githubusercontent.com/jedc4xer/abs_group_assessment/main/data/state_codes.csv'
    state_codes = pd.read_csv(state_code_path)
    
    def clean_for_map(df, target = 'industry'):
        if target not in ['gender','race_group_label','industry']:
            possible = ", ".join(['gender','race_group_label','industry'])
            print(f'You must choose one of: {possible}')
            return
        label = {'gender': 'Owner Gender','race_group_label': 'Owner Race','industry':'Industry'}[target]

        subset = df[(df.payann != 0) & (df.gender != 'Total') & (df.race_group_label != 'Total') & (df.industry_code != '00')]
        subset = subset[['name','geo_id',target,'payann','firmpdemp','emp']]
        subset['payann'] = subset['payann'] * 1000
        subset['avg_pay'] = subset.apply(lambda row: row.payann/row.emp, axis = 1)

        subset_agg = subset.groupby(
            by = ['name']
                )[['avg_pay']].agg(max_pay = ('avg_pay','max'))

        subset_agg.reset_index(inplace = True)

        grouped = pd.merge(subset, subset_agg, left_on = 'name', right_on = 'name', how = 'left')

        grouped = grouped[(grouped.avg_pay == grouped.max_pay)].reset_index(drop = True)
        grouped = grouped.sort_values(by = 'name').reset_index(drop = True)
        grouped = grouped.drop_duplicates().reset_index(drop = True)

        grouped = pd.merge(grouped,state_codes, left_on = 'name', right_on = 'State', how = 'left')

        grouped = grouped[['name',target,'avg_pay','Postal Abbr.']]

        diffs = set(subset_agg.name.tolist()) - set(grouped.name.tolist())
        if len(diffs) > 0:
            print('There are some states missing.')
        return grouped, target, label
    
    def plot_maps(grouped, target):
        grouped, target, label = clean_for_map(grouped, target)

        if target == 'gender':
            color_scheme = px.colors.qualitative.Plotly[2:]
        else:
            color_scheme = px.colors.qualitative.Bold

        customdata = np.stack((grouped['name'],grouped[target]),axis = -1)

        mapfig = px.choropleth(grouped, 
                               title = f'Highest Average Pay based on {label}',
                               locations = 'Postal Abbr.',
                               locationmode = "USA-states",
                               scope = 'usa',
                               color = target,
                               hover_data = ['name',target,'avg_pay'],
                               color_discrete_sequence = color_scheme,
                               height = 600,
                               width = 1300,
                               )

        mapfig.add_annotation(
            text = (f"Created by: Jed Dryer<br>{today}<br>Source: US Census - 2019 ABS"),
            showarrow=False,
            x = .54,
            y = .01,
            xref='paper',
            yref='paper' ,
            xanchor='left',
            yanchor='bottom',
            xshift=-1,
            yshift=-3,
            font=dict(size=10, color="grey"),
            align="right",
            )

        mapfig.update_layout(margin={"r":3,"t":30,"l":0,"b":10}, dragmode = False)
        mapfig.update_layout(title_y = 0.95, title_x = 0.25, title_font_size = 18,
                            legend = dict(x=0.9,y=0.6))
        mapfig.update_traces(hovertemplate = '<b>State: %{customdata[0]}</b><br>Value: %{customdata[1]}')
        mapfig.update_geos(resolution=50,showlakes=True, lakecolor="Lightblue", subunitcolor = 'black')
        #save_label = "".join(_.lower() for _ in label if _.isalpha())
        #mapfig.write_html(f'highest_avg_pay_{save_label}.html')
        #mapfig.show()
        return mapfig

    reasons = ['gender','race_group_label','industry']
    maps = []
    for reason in reasons:
        maps.append(plot_maps(comp_sum_df, reason))

    with open('company_pay_maps.html', 'w') as mapfile:
        for fig in maps:
            mapfile.write(fig.to_html(full_html = False, include_plotlyjs='cdn'))
        mapfile.close()
    
    print("Jed's Visuals Successfully Exported to HTML")

# run my visual function and pass in the dataframe
jeds_visuals() # <--- This is the code to create my visuals

Jed's Visuals Successfully Exported to HTML


In [18]:
# I suggest wrapping your entire visualization process in a function while we work on integrating. 

# For example. 

def jeds_visuals(needed_data_frames):
    # muddy_code
    
    
    pass


In [19]:
# I suggest wrapping your entire visualization process in a function while we work on integrating. 

# For example. 

def jeds_visuals(needed_data_frames):
    # muddy_code
    pass


In [21]:
# I suggest wrapping your entire visualization process in a function while we work on integrating. 

# For example. 

def jeds_visuals(needed_data_frames):
    # muddy_code
    pass